# EXERCISE 3.13

foldRightでfoldLeftが作れるか。逆は？

わからないので、答えを見る。ちなみに、答えを見てもわからない。

```Scala
  /*
  The implementation of `foldRight` in terms of `reverse` and `foldLeft` is a common trick for avoiding stack overflows
  when implementing a strict `foldRight` function as we've done in this chapter. (We'll revisit this in a later chapter,
  when we discuss laziness).
  The other implementations build up a chain of functions which, when called, results in the operations being performed
  with the correct associativity. We are calling `foldRight` with the `B` type being instantiated to `B => B`, then
  calling the built up function with the `z` argument. Try expanding the definitions by substituting equals for equals
  using a simple example, like `foldLeft(List(1,2,3), 0)(_ + _)` if this isn't clear. Note these implementations are
  more of theoretical interest - they aren't stack-safe and won't work for large lists.
  */
  def foldRightViaFoldLeft[A,B](l: List[A], z: B)(f: (A,B) => B): B =
    foldLeft(reverse(l), z)((b,a) => f(a,b))

  def foldRightViaFoldLeft_1[A,B](l: List[A], z: B)(f: (A,B) => B): B =
    foldLeft(l, (b:B) => b)((g,a) => b => g(f(a,b)))(z)

  def foldLeftViaFoldRight[A,B](l: List[A], z: B)(f: (B,A) => B): B =
    foldRight(l, (b:B) => b)((a,g) => b => g(f(b,a)))(z)
```

では、やってみる。

In [8]:
sealed trait List[+A]
case object Nil extends List[Nothing]
case class Cons[+A](head: A, tail: List[A]) extends List[A]

object List {
    def sum(ints: List[Int]): Int =
        ints match {
            case Nil => 0
            case Cons(x, xs) => x + sum(xs)
        }
    
    def apply[A](as: A*): List[A] =
        if (as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))
    
    def tail[A](as: List[A]): List[A] =
        as match {
            case Nil => Nil
            case Cons(h, t) => t
        }
    
    def setHead[A](l: List[A], a: A): List[A] =
        l match {
            case Nil => apply(a)
            case Cons(_, t) => Cons(a, t)
        }
    
    def drop[A](l: List[A], n: Int): List[A] =
        if (n <= 0) l
        else l match {
            case Nil => Nil
            case Cons(_, t) => drop(t, n-1)
        }
    
    def dropWhile[A](l: List[A], f: A => Boolean): List[A] =
        l match {
            case Cons(h, t) => if (f(h)) dropWhile(t, f) else l
            case _ => l
        }
    
    def init[A](l: List[A]): List[A] =
        l match {
            case Nil => sys.error("empty")
            case Cons(_,Nil) => Nil
            case Cons(h,t) => Cons(h, init(t))
        }
 
    def foldRight[A,B](as: List[A], z: B)(f: (A, B) => B): B = // Utility functions
        as match {
          case Nil => z
          case Cons(x, xs) => f(x, foldRight(xs, z)(f))
    }
    
    def length[A](as: List[A]): Int =
        foldRight(as, 0)((x,y) => 1 + y)
    
    @annotation.tailrec
    def foldLeft[A,B](as: List[A], z: B)(f: (B, A) => B): B =
        as match {
            case Nil => z
            case Cons(x, xs) => foldLeft(xs, f(z, x))(f)
        }
    
    def reverse[A](as: List[A]): List[A] =
        foldLeft(as, Nil:List[A])((z, l) => Cons(l, z))
    
    def foldRightViaFoldLeft[A,B](l: List[A], z: B)(f: (A,B) => B): B =
        foldLeft(reverse(l), z)((b,a) => f(a,b))

    def foldRightViaFoldLeft_1[A,B](l: List[A], z: B)(f: (A,B) => B): B =
        foldLeft(l, (b:B) => b)((g,a) => b => g(f(a,b)))(z)

    def foldLeftViaFoldRight[A,B](l: List[A], z: B)(f: (B,A) => B): B =
        foldRight(l, (b:B) => b)((a,g) => b => g(f(b,a)))(z)    
}

val a = List.foldRightViaFoldLeft(List(1,2,3), 0)((_ + _))
val b = List.foldRightViaFoldLeft_1(List(1,2,3), 0)((_ + _))
val c = List.foldLeftViaFoldRight(List(1,2,3), 0)((_ + _))

defined trait List
defined object Nil
defined class Cons
defined object List
a: Int = 6
b: Int = 6
c: Int = 6

後ろ2つがわからないわな。検索すると StackOverFlow にあった。

が、よくわからん。ちょっと別途考える。

http://stackoverflow.com/questions/17136794/foldleft-using-foldright-in-scala

Let's have a look at

```Scala
def foldLeftViaFoldRight[A,B](l: List[A], z: B)(f: (B,A) => B): B = 
  foldRight(l, (b:B) => b)((a,g) => b => g(f(b,a)))(z)
```

(the other fold is similar). The trick is that during the right fold operation, we don't build the final value of type B. Instead, we build a function from B to B. The fold step takes a value of type a: A and a function g: B => B and produces a new function (b => g(f(b,a))): B => B. This function can be expressed as a composition of g with f(_, a):

```Scala
  l.foldRight(identity _)((a,g) => g compose (b => f(b,a)))(z);
```

We can view the process as follows: For each element a of l we take the partial application b => f(b,a), which is a function B => B. Then, we compose all these functions in such a way that the function corresponding to the rightmost element (with which we start the traversal) is at far left in the composition chain. Finally, we apply the big composed function on z. This results in a sequence of operations that starts with the leftmost element (which is at far right in the composition chain) and finishes with the right most one.

Update: As an example, let's examine how this definition works on a two-element list. First, we'll rewrite the function as

```Scala
def foldLeftViaFoldRight[A,B](l: List[A], z: B)
                             (f: (B,A) => B): B =
{
  def h(a: A, g: B => B): (B => B) =
    g compose ((x: B) => f(x,a));
  l.foldRight(identity[B] _)(h _)(z);
}
```

Now let's compute what happens when we pass it List(1,2):

```Scala
List(1,2).foldRight(identity[B] _)(h _)
  = // by the definition of the right fold
h(1, h(2, identity([B])))
  = // expand the inner `h`
h(1, identity[B] compose ((x: B) => f(x, 2)))
  =
h(1, ((x: B) => f(x, 2)))
  = // expand the other `h`
((x: B) => f(x, 2)) compose ((x: B) => f(x, 1))
  = // by the definition of function composition
(y: B) => f(f(y, 1), 2)
```

Applying this function to z yields
```Scala
f(f(z, 1), 2)
```
as required.